### Extract value from MUSTANG-2 data (this is correct)

In [1]:
from astropy import coordinates
from astropy import units as u
coordinates.SkyCoord('17:46:10.63 -28:42:17.8', unit=(u.h, u.deg), frame='icrs').galactic

<SkyCoord (Galactic): (l, b) in deg
    (0.26096684, 0.01615326)>

In [2]:
import numpy as np
# What is the mean brightness in a 62.9" diameter aperture centered on our pointing in MGPS?
# REMEMBER: CARTA regions take semi-axes (radii)!
bmaj = (2.500000000000E-03*u.deg).to(u.arcsec)
bmin = (2.500000000000E-03*u.deg).to(u.arcsec)
fwhm_to_sigma = 1./(8*np.log(2))**0.5
beam_area = 2.*np.pi*(bmaj*bmin*fwhm_to_sigma**2)
print(beam_area)
freq = 90*u.GHz
equiv = u.brightness_temperature(freq)
(1.756621805784e-2*u.Jy/beam_area).to(u.K, equivalencies=equiv) # We started with Jy, and incorporated beam info

91.78029287200066 arcsec2


<Quantity 0.03272067 K>

In [3]:
beam_area.to(u.sr)

<Quantity 2.15724352e-09 sr>

In [4]:
# Try converting MUSTANG-2 Jy/beam to flux in Jy
(1.756621805784e-2*u.Jy/u.beam)/(beam_area/u.beam)

<Quantity 0.00019139 Jy / arcsec2>

In [5]:
FOV_area = np.pi*(62.9/2*u.arcsec)**2
FOV_area/beam_area

<Quantity 33.85647453>

### Now from the MUSTANG-2 + Planck data

In [6]:
# What is the mean brightness in a 62.9" diameter aperture centered on our pointing in MGPS?
# REMEMBER: CARTA regions take semi-axes (radii)!
bmaj = (2.777777777778E-03*u.deg).to(u.arcsec)
bmin = (2.777777777778E-03*u.deg).to(u.arcsec)
fwhm_to_sigma = 1./(8*np.log(2))**0.5
beam_area = 2.*np.pi*(bmaj*bmin*fwhm_to_sigma**2)
print(beam_area)
freq = 90*u.GHz
equiv = u.brightness_temperature(freq)
(4.089063490269e-2*u.Jy/beam_area).to(u.K, equivalencies=equiv) # We started with Jy, and incorporated beam info

113.30900354569796 arcsec2


<Quantity 0.0616954 K>

### Now figure out continuum source contribution from Walker+2021
#### The below code is not right; you can't sum Jy/beam; you have to take the average

In [36]:
## What is the "summed up flux" of the continuum sources found by Walker+2021?
integrated_fluxes = [47.87,#12.10,14.34,
                     2.15,0.69,0.95,2.63,1.48,0.62,0.41,3.27,1.61,1.63,1.06,1.68,7.17,2.34,1.34,0.76]*u.mJy
radii = [4847,#1300,1300,
         1475,1275,1543,1711,1682,1179,945,1687,1236,1161,945,1139,2613,2206,1568,1302]*u.AU
# Convert physical sizes back to angular diams using provided distance 8.1 kpc; small angle equation
angular_diams = (206265*(radii.to(u.pc)*2)/(8100*u.pc))*u.arcsec
angular_areas = np.pi*(angular_diams/2)**2
# Divide each integrated flux by angular area of source to get average surface brightness in mJy/arcsec^2
avg_surf_brightnesses = integrated_fluxes/angular_areas
# Then convert the average surface brightnesses in mJy/arcsec^2 to mJy/beam using the beam area, given in the paper
# BUT I DON'T NEED THIS, IT GIVES THE SAME ANSWER WITH OR WITHOUT IT
# bmaj = 0.17*u.arcsec
# bmin = 0.12*u.arcsec
# fwhm_to_sigma = 1./(8*np.log(2))**0.5
# beam_area = 2.*np.pi*(bmaj*bmin*fwhm_to_sigma**2)

# avg_surf_brightnesses_perbeam = (avg_surf_brightnesses*beam_area)/u.beam
# print(avg_surf_brightnesses_perbeam)

# freq = 230*u.GHz
avg_brightness_temps = avg_surf_brightnesses.to(u.K, u.brightness_temperature(freq))  
np.mean(avg_brightness_temps)

<Quantity 3.0651238 K>

In [37]:
# Nazar's method
## What is the summed up flux of the continuum sources found by Walker+2021?
integrated_fluxes = [47.87,#12.10,14.34,
                     2.15,0.69,0.95,2.63,1.48,0.62,0.41,3.27,1.61,1.63,1.06,1.68,7.17,2.34,1.34,0.76]*u.mJy
radii = [4847,#1300,1300,
         1475,1275,1543,1711,1682,1179,945,1687,1236,1161,945,1139,2613,2206,1568,1302]*u.AU
# Convert physical sizes back to angular diams using provided distance 8.1 kpc; small angle equation
angular_diams = (206265*(radii.to(u.pc)*2)/(8100*u.pc))*u.arcsec
angular_areas = np.pi*(angular_diams/2)**2 

bmaj = 0.17*u.arcsec
bmin = 0.12*u.arcsec
fwhm_to_sigma = 1./(8*np.log(2))**0.5
dans_beam_area = 2.*np.pi*(bmaj*bmin*fwhm_to_sigma**2)
freq = 230*u.GHz
brightness_temps = (integrated_fluxes/((angular_areas/dans_beam_area).to(u.dimensionless_unscaled))/dans_beam_area).to(u.K, equivalencies=u.brightness_temperature(freq))
np.mean(brightness_temps)

# Divide each integrated flux by angular area of source to get average surface brightness in mJy/arcsec^2

# avg_surf_brightnesses = integrated_fluxes/angular_areas

# avg_brightness_temps = avg_surf_brightnesses.to(u.K, u.brightness_temperature(freq))  
# np.sum(avg_brightness_temps)

<Quantity 0.46932898 K>